### this setcion code is for reading picture both smo and nosmo ,and convert them to tfRecode format! ###!!


In [ ]:
#https://kwotsin.github.io/tech/2017/02/11/transfer-learning.html

#================ DATASET INFORMATION ======================
#State dataset directory where the tfrecord files are located
dataset_dir = '.'

#State where your log file is at. If it doesn't exist, create it.
log_dir = './out001/'

#State where your checkpoint file is
checkpoint_file = './inception_resnet_v2_2016_08_30.ckpt'

#State the image size you're resizing your images to. We will use the default inception size of 299.
image_size = 299

#State the number of classes to predict:
num_classes = 2

#State the labels file and read it

labels_file ='./labels.txt'
labels = open(labels_file, 'r')

#Create a dictionary to refer each label to their string name
labels_to_name = {}

# a dictionary of datasets_data_dir name
#datasets_data_dir = '/home/aewin/work/anaconda3/code/models/research/slim/smorking'
datasets_data_dir = '/home/aewin/work/anaconda3/code/V001/smodata/t'

#================= TRAINING INFORMATION ==================
#State the number of epochs to train
num_epochs = 15 #3 mins every time

#State your batch size
batch_size = 32



#Create a dictionary to refer each label to their string name
labels_to_name = {}
for line in labels:
    label, string_name = line.split(':')
    string_name = string_name[:-1] #Remove newline
    labels_to_name[int(label)] = string_name



#Learning rate information and configuration (Up to you to experiment)
initial_learning_rate = 0.0002
learning_rate_decay_factor = 0.7
num_epochs_before_decay = 2
file_pattern = 'smotfrecord_train_%s.tfrecord'


items_to_descriptions = {
    'image': 'A 3-channel RGB coloured flower image that is either tulips, sunflowers, roses, dandelion, or daisy.',
    'label': 'A label that is as such -- 0:nosmo, 1:smo'
}


In [ ]:

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib import slim
import os
import time

def get_split(split_name, dataset_dir, file_pattern=file_pattern):
    '''
    Obtains the split - training or validation - to create a Dataset class for feeding the examples into a queue later on. This function will
    set up the decoder and dataset information all into one Dataset class so that you can avoid the brute work later on.
    Your file_pattern is very important in locating the files later. 
    INPUTS:
    - split_name(str): 'train' or 'validation'. Used to get the correct data split of tfrecord files
    - dataset_dir(str): the dataset directory where the tfrecord files are located
    - file_pattern(str): the file name structure of the tfrecord files in order to get the correct data
    OUTPUTS:
    - dataset (Dataset): A Dataset class object where we can read its various components for easier batch creation later.
    '''

    #First check whether the split_name is train or validation
    if split_name not in ['train', 'validation']:
        raise ValueError('The split_name %s is not recognized. Please input either train or validation as the split_name' % (split_name))

    #Create the full path for a general file_pattern to locate the tfrecord_files
    file_pattern_path = file_pattern

    #Count the total number of examples in all of these shard
    num_samples = 0
    '''
    file_pattern_for_counting = '200label_' + split_name
    tfrecords_to_count = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir) if file.startswith(file_pattern_for_counting)]
    print(file_pattern_for_counting)
    '''
    tfrecords_to_count = [os.path.join(dataset_dir,file) for file in os.listdir(dataset_dir)]
         
    for tfrecord_file in tfrecords_to_count:
        for record in tf.python_io.tf_record_iterator(tfrecord_file):
            num_samples += 1

    
    print(tfrecords_to_count)
    
    #Create a reader, which must be a TFRecord reader in this case
    reader = tf.TFRecordReader

    #Create the keys_to_features dictionary for the decoder
    keys_to_features = {
      'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
      'image/format': tf.FixedLenFeature((), tf.string, default_value='jpg'),
      'image/class/label': tf.FixedLenFeature(
          [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }

    #Create the items_to_handlers dictionary for the decoder.
    items_to_handlers = {
    'image': slim.tfexample_decoder.Image(),
    'label': slim.tfexample_decoder.Tensor('image/class/label'),
    }

    #Start to create the decoder
    decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

    #Create the labels_to_name file
    labels_to_name_dict = labels_to_name
    
    print(tfrecords_to_count)

    
    
    #Actually create the dataset
    dataset = slim.dataset.Dataset(
        data_sources = tfrecords_to_count,
        decoder = decoder,
        reader = reader,
        num_readers = 4,
        num_samples = num_samples,
        num_classes = num_classes,
        labels_to_name = labels_to_name_dict,
        items_to_descriptions = items_to_descriptions)
    
    
    
    
    print(dataset.data_sources)
    return dataset


#get_split('train', datasets_data_dir, file_pattern=file_pattern)
#file_pattern = 'smotfrecord_validation_%s.tfrecord'
#get_split('validation', datasets_data_dir, file_pattern=file_pattern)

In [ ]:
from datasets import flowers
import setdate #from fllower.py
import inception_preprocessing


def load_batch(dataset, batch_size, height=image_size, width=image_size, is_training=True):
    '''
    Loads a batch for training.

    INPUTS:
    - dataset(Dataset): a Dataset class object that is created from the get_split function
    - batch_size(int): determines how big of a batch to train
    - height(int): the height of the image to resize to during preprocessing
    - width(int): the width of the image to resize to during preprocessing
    - is_training(bool): to determine whether to perform a training or evaluation preprocessing

    OUTPUTS:
    - images(Tensor): a Tensor of the shape (batch_size, height, width, channels) that contain one batch of images
    - labels(Tensor): the batch's labels with the shape (batch_size,) (requires one_hot_encoding).

    '''
    #First create the data_provider object
    '''   
    参数说明如下：
    dataset：Dataset 类实例
    num_readers：并行阅读器数量
    reader_kwargs=None：阅读器关键配置字典
    shuffle：是否打乱
    num_epochs：每个数据源被读取的次数，如果设为None数据将会被无限循环的读取
    common_queue_capacity：读取数据队t列的容量，默认为256
    common_queue_min：读取数据队列的最小容量
    record_key：（不是很理解）
    seed=None：打乱是的种子
    scope=None：范围
    ''' 
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset,
        common_queue_capacity = 24 + 3 * batch_size,
        common_queue_min = 24)

    #Obtain the raw image using the get method
    raw_image, label = data_provider.get(['image', 'label'])

    '''
    with tf.Session() as sess:    
        with slim.queues.QueueRunners(sess):
            for i in range(7):
                np_image, np_label = sess.run([raw_image, label])
                print(np_label)
                height, width, _ = np_image.shape
                #class_name = name = dataset.labels_to_names[np_label]
                
                plt.figure()
                plt.imshow(np_image)
                #plt.title('%s, %d x %d' % (name, height, width))
                plt.axis('off')
                plt.show()

    '''
    #Perform the correct preprocessing for this image depending if it is training or evaluating
    image = inception_preprocessing.preprocess_image(raw_image, height, width, is_training)
    
    #As for the raw images, we just do a simple reshape to batch it up
    raw_image = tf.expand_dims(raw_image, 0)
    raw_image = tf.image.resize_nearest_neighbor(raw_image, [height, width])
    raw_image = tf.squeeze(raw_image)

    #Batch up the image by enqueing the tensors internally in a FIFO queue and dequeueing many elements with tf.train.batch.
    images, raw_images, labels = tf.train.batch(
        [image, raw_image, label],
        batch_size = batch_size,
        num_threads = 4,
        capacity = 4 * batch_size,
        allow_smaller_final_batch = True)

    return images, raw_images, labels

load_batch(get_split('train', datasets_data_dir, file_pattern=file_pattern)
, batch_size=batch_size)


In [ ]:
import datetime
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))


In [ ]:

#from datasets import flowers

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib import slim
import os
from inception_resnet_v2v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
import time
#https://blog.gtwang.org/programming/tensorflow-read-write-tfrecords-data-format-tutorial/
#实例化了一个类，一个用于 tensorflow 计算和表示用的数据流图
'''  WHY using  tf.Graph().as_default():!!!!!!!!!!!!!!!
   Since a default graph is always registered, every op and variable is placed into the default graph. 
   The statement of  tf.Graph().as_default(): , however,creates a new graph and places everything (declared inside its scope) into this graph.
   If the graph is the only graph, it's useless. 
   But it's a good practice because if you start to work with many graphs it's easier to understand where ops and 
   vars are placed. 
   Since this statement costs you nothing, it's better to write it anyway. 
   Just to be sure that if you refactor the code in the future, 
   the operations defined belong to the graph you choose initially
'''
def run():
    with tf.Graph().as_default(): 
  
        
        #指定獲取有“train”的資料
        #dataset = get_split2('train',  './tf_train/','smorking_train_*.tfrecords')
        #dataset = get_split('train',  './tf_train/','smorking_train_*.tfrecords')
        
        dataset = get_split('train', datasets_data_dir, file_pattern=file_pattern)
        image, _, labels = load_batch(dataset, batch_size=batch_size)

 
        #Know the number steps to take before decaying the learning rate and batches per epoch
        num_batches_per_epoch = int(dataset.num_samples / batch_size)
        num_steps_per_epoch = num_batches_per_epoch #Because one step is one batch processed
        decay_steps = int(num_epochs_before_decay * num_steps_per_epoch)

        #Create the model inference
        with slim.arg_scope(inception_resnet_v2_arg_scope()):

            logits, end_points = inception_resnet_v2(image, num_classes = dataset.num_classes, is_training = True)

        #Define the scopes that you want to exclude for restoration
        exclude = ['InceptionResnetV2/Logits', 'InceptionResnetV2/AuxLogits']
        variables_to_restore = slim.get_variables_to_restore(exclude = exclude)

        #Perform one-hot-encoding of the labels (Try one-hot-encoding within the load_batch function!)
        one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)

        #Performs the equivalent to tf.nn.sparse_softmax_cross_entropy_with_logits but enhanced with checks
        loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = logits)
        total_loss = tf.losses.get_total_loss()    #obtain the regularization losses as well

        #Create the global step for monitoring the learning_rate and training.
        global_step = get_or_create_global_step()
        #Define your exponentially decaying learning rate
        lr = tf.train.exponential_decay(
            learning_rate = initial_learning_rate,
            global_step = global_step,
            decay_steps = decay_steps,
            decay_rate = learning_rate_decay_factor,
            staircase = True)

        #Now we can define the optimizer that takes on the learning rate
        optimizer = tf.train.AdamOptimizer(learning_rate = lr)

        #Create the train_op.
        train_op = slim.learning.create_train_op(total_loss, optimizer)

        #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
        predictions = tf.argmax(end_points['Predictions'], 1)
        probabilities = end_points['Predictions']
        accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, labels)
        metrics_op = tf.group(accuracy_update, probabilities)


        #Now finally create all the summaries you need to monitor and group them into one summary op.
        tf.summary.scalar('losses/Total_Loss', total_loss)
        tf.summary.scalar('accuracy', accuracy)
        tf.summary.scalar('learning_rate', lr)
        my_summary_op = tf.summary.merge_all()
        

        #Now we need to create a training step function that runs both the train_op, metrics_op and updates the global_step concurrently.
        def train_step(sess, train_op, global_step):
            '''
            Simply runs a session for the three arguments provided and gives a logging on the time elapsed for each global step
            '''
            #Check the time for each sess run
            start_time = time.time()
            total_loss, global_step_count, _ = sess.run([train_op, global_step, metrics_op])
            time_elapsed = time.time() - start_time

            #Run the logging to print some results
            print('global step %s: loss: %.4f (%.2f sec/step)' %( global_step_count, total_loss, time_elapsed))
            return total_loss, global_step_count

        #Now we create a saver function that actually restores the variables from a checkpoint file in a sess
        #write_version=tf.train.SaverDef.V1# this is  for V1
        saver = tf.train.Saver(variables_to_restore)
        
        def restore_fn(sess):
            return saver.restore(sess, checkpoint_file)

        #Define your supervisor for running a managed session. Do not run the summary_op automatically or else it will consume too much memory
        sv = tf.train.Supervisor(logdir = log_dir, summary_op = None, init_fn = restore_fn)

         
        #Run the managed session
        with sv.managed_session() as sess:


            for step in range(num_steps_per_epoch * num_epochs):
                print(step)
                #At the start of every epoch, show the vital information:
                if (step % num_batches_per_epoch) == 0:
                    print('Epoch %s/%s' %(step/num_batches_per_epoch + 1, num_epochs))
                    learning_rate_value, accuracy_value = sess.run([lr, accuracy])
                    print('Current Learning Rate: %s' %(learning_rate_value))
                    print('Current Streaming Accuracy: %s' %(accuracy_value))

                    # optionally, print your logits and predictions for a sanity check that things are going fine.
                    logits_value, probabilities_value, predictions_value, labels_value = sess.run([logits, probabilities, predictions, labels])
                    print( 'logits: \n', logits_value)
                    print( 'Probabilities: \n', probabilities_value)
                    print( 'predictions: \n', predictions_value)
                    print( 'Labels:\n:', labels_value)

                #Log the summaries every 10 step.
                if (step % 10) == 0:
                    print('step % 10==0')
                    loss, _ = train_step(sess, train_op, sv.global_step)
                    summaries = sess.run(my_summary_op)
                    sv.summary_computed(sess, summaries)
                    
                #If not, simply run the training step
                else:
                    loss, _ = train_step(sess, train_op, sv.global_step)

            #We log the final training loss and accuracy
            print('Final Loss: %s' %loss)
            print('Final Accuracy: %s' %sess.run(accuracy))
                          

            #Once all the training has been done, save the log files and checkpoint model
            print('Finished training! Saving model to disk now.')
            
            #sv.saver.save(sess, './model.ckpt', global_step = sv.global_step)
            
            sv.saver.save(sess, sv.save_path, global_step = sv.global_step)# this is  for V2
            

run()     

#import tensorflow as tf
#from tensorflow.python.framework import graph_io
#frozen = tf.graph_util.convert_variables_to_const#ants(sess, sess.graph_def, ["<name_of_the_output_node>"])
#graph_io.write_graph(frozen, './', 'inference_graph.pb', as_text=False)





In [ ]:
import datetime
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))


In [ ]:
! example :: freeze_graph.py --input_graph=./code/V12/graph.pbtxt  --input_checkpoint=./code/V12/model.ckpt-0 --input_binary=false   --output_graph=./code/V12/frozen_graph.pb --output_node_names=InceptionResnetV2/Logits/Predictions
# EXample "" mvNCCompile -s 12  ./frozen_model_inception_resnet_v2.pb -in=Placeholder_only -on=InceptionResnetV2/Logits/Predictions



## Validation

In [ ]:
import tensorflow as tf
from tensorflow.python.platform import tf_logging as logging
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
import inception_preprocessing
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
import time
import os
#from train_flowers import load_batch
import matplotlib.pyplot as plt
plt.style.use('ggplot')
slim = tf.contrib.slim

#State your log directory where you can retrieve your model
log_dir = './out001/'

#Create a new evaluation log directory to visualize the validation process
log_eval = './log_eval_test'

#State the dataset directory where the validation set is found
dataset_dir = datasets_data_dir

#State the batch_size to evaluate each time, which can be a lot more than the training batch
batch_size = 100

#State the number of epochs to evaluate
num_epochs = 50

#Get the latest checkpoint file
checkpoint_file = tf.train.latest_checkpoint(log_dir)
file_pattern = 'smotfrecord_validation_%s.tfrecord'

def main():
    #Create log_dir for evaluation information
    if not os.path.exists(log_eval):
        os.mkdir(log_eval)

    #Just construct the graph from scratch again
    with tf.Graph().as_default() as graph:
        tf.logging.set_verbosity(tf.logging.INFO)
        #Get the dataset first and load one batch of validation images and labels tensors. Set is_training as False so as to use the evaluation preprocessing
      
        dataset = get_split('validation', datasets_data_dir, file_pattern=file_pattern)
        print(dataset.data_sources)
        
        #image, raw_images, labels = load_batch(dataset, batch_size=batch_size)
        image, raw_images, labels = load_batch(dataset, batch_size = batch_size, is_training = False)
 
        
        #Create some information about the training steps
        num_batches_per_epoch = dataset.num_samples / batch_size
        num_steps_per_epoch = num_batches_per_epoch

        #Now create the inference model but set is_training=False
        with slim.arg_scope(inception_resnet_v2_arg_scope()):
            logits, end_points = inception_resnet_v2(image, num_classes = num_classes, is_training = False)

        # #get all the variables to restore from the checkpoint file and create the saver function to restore
        variables_to_restore = slim.get_variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)
        def restore_fn(sess):
            return saver.restore(sess, checkpoint_file)

        #Just define the metrics to track without the loss or whatsoever
        predictions = tf.argmax(end_points['Predictions'], 1)
        accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, labels)
        metrics_op = tf.group(accuracy_update)

        #Create the global step and an increment op for monitoring
        global_step = get_or_create_global_step()
        global_step_op = tf.assign(global_step, global_step + 1) #no apply_gradient method so manually increasing the global_step
        
        #Create a evaluation step function
        def eval_step(sess, metrics_op, global_step):
           
            #Simply takes in a session, runs the metrics op and some logging information.
           
            start_time = time.time()
            _, global_step_count, accuracy_value = sess.run([metrics_op, global_step_op, accuracy])
            time_elapsed = time.time() - start_time

            #Log some information
            print('Global Step %s: Streaming Accuracy: %.4f (%.2f sec/step)' %(global_step_count, accuracy_value, time_elapsed))

            return accuracy_value


        #Define some scalar quantities to monitor
        tf.summary.scalar('Validation_Accuracy', accuracy)
        my_summary_op = tf.summary.merge_all()

        #Get your supervisor
        sv = tf.train.Supervisor(logdir = log_eval, summary_op = None, saver = None, init_fn = restore_fn)

        #Now we are ready to run in one session
        print(num_steps_per_epoch)
        print(num_epochs)
                
        with sv.managed_session() as sess:
            for step in range( num_epochs):
                
                sess.run(sv.global_step)
                #print vital information every start of the epoch as always
                if (step % num_batches_per_epoch) == 0:
                    print('Epoch: %s/%s' %(step / num_batches_per_epoch + 1, num_epochs))
                    
                    print('Current Streaming Accuracy: %f' %(sess.run(accuracy)))
                    
                #Compute summaries every 10 steps and continue evaluating
                if (step % 10) == 0:
                    eval_step(sess, metrics_op = metrics_op, global_step = sv.global_step)
                    summaries = sess.run(my_summary_op)
                    sv.summary_computed(sess, summaries)
                    

                #Otherwise just run as per normal
                else:
                    eval_step(sess, metrics_op = metrics_op, global_step = sv.global_step)

            #At the end of all the evaluation, show the final accuracy
            
             #Now we want to visualize the last batch's images just to see what our model has predicted
            raw_images, labels, predictions = sess.run([raw_images, labels, predictions])
            for i in range(99):
                
                image, label, prediction = raw_images[i], labels[i], predictions[i]
                print('index = %d' %(i))
               
                print('labels= %d predictions= %d ' %(labels[i] ,predictions[i]))
                
                prediction_name = dataset.labels_to_name[prediction],
                label_name = dataset.labels_to_name[label]
                
          
                print(label)
                print(prediction)
                label_name = dataset.labels_to_name[label]
                print('label_name: %s' %(label_name))
                prediction_name = dataset.labels_to_name[prediction]
                print('Prediction: %s' %(prediction_name))
               
                #text =('Prediction: %s \n Ground Truth: %s' %(prediction_name, label_name))
                print('Prediction: %s \n Ground Truth: %s' %(prediction_name, label_name))
                
                img_plot = plt.imshow(image)

                #Set up the plot and hide axes
                #plt.title(text)
                img_plot.axes.get_yaxis().set_ticks([])
                img_plot.axes.get_xaxis().set_ticks([])
                plt.show()

            
            print('Model evaluation has completed! Visit TensorBoard for more information regarding your evaluation.')
       
if __name__ == '__main__':
  main()

In [ ]:
from __future__ import print_function
import tensorflow as tf
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
from tensorflow.python.framework import graph_util
slim = tf.contrib.slim

def CreatePbFile():
    image_size = 299
    num_classes = 2

    
    checkpoint_file = tf.train.latest_checkpoint(log_dir)

    with tf.Graph().as_default() as graph:
        
        images = tf.placeholder(shape=[None, image_size, image_size, 3], dtype=tf.float32, name = 'Placeholder_only')

        with slim.arg_scope(inception_resnet_v2_arg_scope()):
            logits, end_points = inception_resnet_v2(images, num_classes = num_classes, is_training = False)

        variables_to_restore = slim.get_variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)

        #Setup graph def
        input_graph_def = graph.as_graph_def()
        output_node_names = "InceptionResnetV2/Logits/Predictions"
        output_graph_name = "./frozen_model_inception_resnet_v2_1218.pb"

        with tf.Session() as sess:
            saver.restore(sess, checkpoint_file)

            #Exporting the graph
            print ("Exporting graph...")
            output_graph_def = graph_util.convert_variables_to_constants(
                sess,
                input_graph_def,
                output_node_names.split(","))

            with tf.gfile.GFile(output_graph_name, "wb") as f:
                f.write(output_graph_def.SerializeToString())

CreatePbFile()    